# 4.0 Computer Vision

In [ ]:
import pandas as pd
import numpy as np
import torch as tc
from torch import nn
import torchvision
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

## 4.1 Getting our datsets

#### Please do not run the code below more than once

In [ ]:
train_data = datasets.FashionMNIST(
    root="DATA", train=True, download=True, transform=ToTensor(), target_transform=None
)

test_data = datasets.FashionMNIST(
    root="DATA", train=False, download=True, transform=ToTensor(), target_transform=None
)

#### Please do not run the code above more than once

In [ ]:
train_data
# seeing the first data
image,label = train_data[0]
image,label

In [ ]:
test_data

In [ ]:
classnames = train_data.classes
classnames

In [ ]:
class_to_idx = train_data.class_to_idx
class_to_idx

## 4.2 Visualising random sample data

In [ ]:
print(f"Image Shape: {image.shape}")
plt.imshow(image.squeeze())
plt.title(label)

In [ ]:
# gray scale
plt.imshow(image.squeeze(),cmap="gray")
plt.title(classnames[label])
plt.axis(False)

In [ ]:
# visualising random images
tc.manual_seed(42)
fig = plt.figure(figsize=(9,9))
rows,cols = 4,4
for i in range(1,rows*cols+1):
  random_idx = tc.randint(0,len(train_data),size=[1]).item()
  # print(random_idx)
  img,lab = train_data[random_idx]
  fig.add_subplot(rows,cols,i)
  plt.imshow(img.squeeze(),cmap="gray")
  plt.title(classnames[lab])
  plt.axis(False)

### 4.2.1 Preparing Data Loader

In [ ]:
from torch.utils.data import DataLoader

# batch size hyper parameter
BATCH_SIZE = 32

# turn dataset into iterable
train_dataloader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(dataset=test_data, batch_size=BATCH_SIZE, shuffle=False)

train_dataloader,test_dataloader

In [ ]:
print(f"DataLoaders: {train_dataloader,test_dataloader}")
print(f"Length of train data loader: {len(train_dataloader)} batches of {BATCH_SIZE}...")
print(f"Length of test data loader: {len(test_dataloader)} batches of {BATCH_SIZE}...")

In [ ]:
# checking training data loader
train_features_batch, train_label_batch = next(iter(train_dataloader))
train_features_batch.shape,train_label_batch.shape

In [ ]:
# show a sample
tc.manual_seed(42)
random_idx = tc.randint(0,len(train_features_batch),size=[1]).item()
img, lab = train_features_batch[random_idx],train_label_batch[random_idx]
plt.imshow(img.squeeze(),cmap="gray")
plt.title(classnames[lab])
plt.axis(False)
print(f"Image size: {img.shape}")
print(f"Label: {lab}, Label size: {lab.shape}")

## 4.3 Creating a model

In [ ]:
# building a baseline model
flatten_model = nn.Flatten()

# get a single sample
x = train_features_batch[0]
x.shape
# flatten x
output = flatten_model(x)
print(f"Shape before flattening: {x.shape}\nShape after flattening: {output.shape}")

In [ ]:
class FashionMNISTModelV0(nn.Module):
  def __init__(self,input_shape:int,hidden_unit:int,output_shpe:int):
    super().__init__()
    self.layer_stack = nn.Sequential(
      nn.Flatten(),
      nn.Linear(in_features=input_shape,out_features=hidden_unit),
      nn.Linear(in_features=hidden_unit,out_features=output_shpe)
    )
    
  def forward(self,x):
    return self.layer_stack(x)

In [ ]:
tc.manual_seed(42)
model0 = FashionMNISTModelV0(input_shape=784,hidden_unit=10,output_shpe=len(classnames))

model0

### 4.3.1 Setup loss function and optimiser

In [ ]:
# import accuracy functions
from helper import accuracy_fn

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimiser = tc.optim.SGD(params=model0.parameters(),lr=0.1)

In [ ]:
# setting up timer function
from timeit import default_timer as timer


def print_train_time(start: float, end: float, devive: tc.device = None):
    """Prints difference between start time and end time"""
    total_time = end - start
    print(f"Train time on {devive}: {total_time:.3f} seconds")
    return total_time

### 4.3.2 Training loop for our model

In [ ]:
# Calculate accuracy (a classification metric)
def accuracy_fn(y_true, y_pred):
    """Calculates accuracy between truth labels and predictions.

    Args:
        y_true (torch.Tensor): Truth labels for predictions.
        y_pred (torch.Tensor): Predictions to be compared to predictions.

    Returns:
        [torch.float]: Accuracy value between y_true and y_pred, e.g. 78.45
    """
    correct = tc.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred)) * 100
    return acc

In [ ]:
from tqdm.auto import tqdm

# set the seed
tc.manual_seed(42)
start_time = timer()

# set epochs
epochs = 3

# create training and test loop
for epoch in tqdm(range(epochs)):
  print(f"Epoch: {epoch}\n-----")
  # training
  train_loss = 0
  # add a loop to loop through the training batches
  for batch,(X,y) in enumerate(train_dataloader):
    model0.train()
    # 1. forward pass
    y_pred = model0(X)
    # 2. calculate the loss
    loss = loss_fn(y_pred,y)
    train_loss +=loss
    # 3. optimise the zero grad
    optimiser.zero_grad()
    # 4. loss backward
    loss.backward()
    # optimise the step
    optimiser.step()
  
    # print what is happening
    if batch % 400 == 0:
      print(f"Looked at {batch * len(X)}/{len(train_dataloader.dataset)} samples")

  # Divide total train loss by the lenght of the train dataloader
  train_loss /= len(train_dataloader)
  
  ### Testing
  test_loss,test_acc = 0,0
  model0.eval()
  with tc.inference_mode():
    for X_test,y_test in test_dataloader:
      # 1. forward pass
      test_pred = model0(X_test)
      # 2. calculate the loss
      test_loss += loss_fn(test_pred,y_test)
      # 3. calculate accuracy
      test_acc += accuracy_fn(y_true=y_test,y_pred=test_pred.argmax(dim=1))
    # calculate the test loss avg per batch
    test_loss /= len(test_dataloader)
    # calculate the test acc avg per batch
    test_acc /= len(test_dataloader)
    
  # print what is happening
  print(f"\nTrain loss: {train_loss:.4f} | Test loss: {test_loss:.4f}, Test acc: {test_acc:.3f}")
  
# Calculate the training time
end_time = timer()
total_time0 = print_train_time(start=start_time,end=end_time,devive=str(next(model0.parameters()).device))

## 4.4 Making Predictions

In [ ]:
# making some predictions
tc.manual_seed(42)
def eval_model(model:tc.nn.Module,data_loader:tc.utils.data.DataLoader,loss_fn:tc.nn.Module,accuracy_fn):
  """Return a disctionary containing the results of model predicting on data loader."""
  
  loss, acc = 0,0
  model.eval()
  with tc.inference_mode():
    for X,y in data_loader:
      # make predictions
      y_pred = model(X)
      
      # accumulate the loss and acc values per patch
      loss += loss_fn(y_pred,y)
      acc += accuracy_fn(y_true=y,y_pred=y_pred.argmax(dim=1))
    # scalle the loss and acc
    loss /= len(data_loader)
    acc /= len(data_loader)
    
  return {"Model name": model.__class__.__name__,"Model Loss": loss.item(),"Model acc": acc}



In [ ]:
# Calculate model 0 result on our dataset
model0_results = eval_model(model=model0,data_loader=test_dataloader,loss_fn=loss_fn,accuracy_fn=accuracy_fn)

model0_results

## 4.5 Improving through experimentation

### 4.5.1 Building a better model with non linearlity

In [ ]:
class FashionMNISTModelV1(nn.Module):
  def __init__(self,input_shape:int,hidden_units:int,output_shape:int):
    super().__init__()
    self.layer_stack = nn.Sequential(
      nn.Flatten(),
      nn.Linear(in_features=input_shape,out_features=hidden_units),
      nn.ReLU(),
      nn.Linear(in_features=hidden_units,out_features=output_shape),
      nn.ReLU()
    )
  
  def forward(self,x:tc.Tensor):
    return self.layer_stack(x)

In [ ]:
# creating an instanc of model 1
tc.manual_seed(42)
model1 = FashionMNISTModelV1(input_shape=784,hidden_units=10,output_shape=len(classnames))


#### 4.5.1.1 Setting up loss function and optimizer and a training loop

In [ ]:
# loss_fn1 = nn.CrossEntropyLoss()
optimiser1 = tc.optim.SGD(params=model1.parameters(),lr=0.1)

In [ ]:

# set the seed
tc.manual_seed(42)
start_time = timer()

# set epochs
epochs = 3

# create training and test loop
for epoch in tqdm(range(epochs)):
    print(f"Epoch: {epoch}\n-----")
    # training
    train_loss = 0
    # add a loop to loop through the training batches
    for batch, (X, y) in enumerate(train_dataloader):
        model1.train()
        # 1. forward pass
        y_pred = model0(X)
        # 2. calculate the loss
        loss = loss_fn(y_pred, y)
        train_loss += loss
        # 3. optimise the zero grad
        optimiser1.zero_grad()
        # 4. loss backward
        loss.backward()
        # optimise the step
        optimiser1.step()

        # print what is happening
        if batch % 400 == 0:
            print(f"Looked at {batch * len(X)}/{len(train_dataloader.dataset)} samples")

    # Divide total train loss by the lenght of the train dataloader
    train_loss /= len(train_dataloader)

    ### Testing
    test_loss, test_acc = 0, 0
    model1.eval()
    with tc.inference_mode():
        for X_test, y_test in test_dataloader:
            # 1. forward pass
            test_pred = model1(X_test)
            # 2. calculate the loss
            test_loss += loss_fn(test_pred, y_test)
            # 3. calculate accuracy
            test_acc += accuracy_fn(y_true=y_test, y_pred=test_pred.argmax(dim=1))
        # calculate the test loss avg per batch
        test_loss /= len(test_dataloader)
        # calculate the test acc avg per batch
        test_acc /= len(test_dataloader)

        # print what is happening
        print(
            f"\nTrain loss: {train_loss:.4f} | Test loss: {test_loss:.4f}, Test acc: {test_acc:.3f}"
        )

# Calculate the training time
end_time = timer()
total_time1 = print_train_time(
    start=start_time, end=end_time, devive=str(next(model1.parameters()).device)
)

#### 4.5.1.2 Functionising raining anf evaluation loop

In [ ]:
def train_step(model:tc.nn.Module,data_loader:tc.utils.data.DataLoader,loss_fn:tc.nn.Module,optimiser:tc.optim.Optimizer,accuracy_fn):
  """Performs training step with model trying to learn on data loader"""
  
  train_loss,train_acc = 0,0
  # put model in training mode
  model.train()
  for batch,(X,y) in enumerate(data_loader):
    # 1. forward pass
    y_pred = model(X)
    
    # 2. calculate the loss and acc per batch
    loss = loss_fn(y_pred,y)
    train_loss += loss
    train_acc += accuracy_fn(y_true=y,y_pred=y_pred.argmax(dim=1))
    
    # 3. optimize the zero grad
    optimiser.zero_grad()
    
    # 4. Loss backward
    loss.backward()
    
    # 5. optimize the step
    optimiser.step()
    
    # print out whats happening
    # if batch % 400 == 0:
    #   print(f"Looked at {batch * len(X)}/{len(train_dataloader.dataset)} samples")
      
  # Divide total train loss and acc by length of train data loader
  train_loss /= len(data_loader)
  train_acc /= len(data_loader)
  print(f"Train loss: {train_loss:.5f} | Train acc: {train_acc:.2f}%")

In [ ]:
def test_step(model:tc.nn.Module,data_loader:tc.utils.data.DataLoader,loss_fn:tc.nn.Module,accuracy_fn):
  """Performs testing step with model trying to learn on data loader"""
  ### Testing
  test_loss, test_acc = 0, 0
  model.eval()
  with tc.inference_mode():
    for X_test, y_test in data_loader:
      # 1. forward pass
      test_pred = model(X_test)
      # 2. calculate the loss
      test_loss += loss_fn(test_pred, y_test)
      # 3. calculate accuracy
      test_acc += accuracy_fn(y_true=y_test, y_pred=test_pred.argmax(dim=1))
    # calculate the test loss avg per batch
    test_loss /= len(data_loader)
    # calculate the test acc avg per batch
    test_acc /= len(data_loader)

    # print what is happening
    print(
      f"\nTest loss: {test_loss:.4f}, Test acc: {test_acc:.3f}"
    )

In [ ]:
tc.manual_seed(42)
# measuring the time
start_time1 = timer()

# set epochs
epchs = 3
# create optimization and evaluation hooks
for epch in tqdm(range(epchs)):
    print(f"Epoch: {epch}")
    train_step(
        model=model1,
        data_loader=train_dataloader,
        loss_fn=loss_fn,
        optimiser=optimiser1,
        accuracy_fn=accuracy_fn,
    )
    test_step(
        model=model1,
        data_loader=test_dataloader,
        loss_fn=loss_fn,
        accuracy_fn=accuracy_fn,
    )
    
end_time1 = timer()
total_time1 = print_train_time(start=start_time1,end=end_time1,devive="cpu")

In [ ]:
print(model0_results)
print(total_time0)

In [ ]:
# get model1 eval dictionary
model1_results = eval_model(model=model1,data_loader=test_dataloader,loss_fn=loss_fn,accuracy_fn=accuracy_fn)
model1_results

### 4.5.2 Improving our model with Convolutional Neural Network (CNN)

In [ ]:
# creating a CNN
class FashionMNISTModelV2(nn.Module):
  """
  Model architechture that replicate a TinyVGG
  """
  def __init__(self,input_shape:int,hidden_units:int,output_shape:int):
    super().__init__()
    self.conv_block_1 = nn.Sequential(
      nn.Conv2d(in_channels=input_shape,out_channels=hidden_units,kernel_size=3,stride=1,padding=1),
      nn.ReLU(),
      nn.Conv2d(in_channels=hidden_units,out_channels=hidden_units,kernel_size=3,stride=1,padding=1),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2)
    )
    
    self.conv_block_2 = nn.Sequential(
      nn.Conv2d(in_channels=hidden_units,out_channels=hidden_units,kernel_size=3,stride=1,padding=1),
      nn.ReLU(),
      nn.Conv2d(in_channels=hidden_units,out_channels=hidden_units,kernel_size=3,stride=1,padding=1),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2)
    )
    
    self.classifier = nn.Sequential(
      nn.Flatten(),
      nn.Linear(in_features=hidden_units*0,out_features=output_shape)
    )
  
  def forward(self,x):
    x = self.conv_block_1(x)
    print(x.shape)
    x = self.conv_block_2(x)
    print(x.shape)
    x = self.classifier(x)
    print(x.shape)
    return x

In [ ]:
tc.manual_seed(42)
model2 = FashionMNISTModelV2(input_shape=1,hidden_units=10,output_shape=len(classnames))

#### 4.5.2.1 Stepping through nn.Conv2d()